This notebook will compare the metadata files created by the SPGC and the pg_catalog.csv from project gutenberg
Then, it will make a dataset

In [1]:
import numpy as np
import pandas as pd
import os, sys
import glob

from collections import Counter
import matplotlib.pyplot as plt

import misc_utils.dataset_filtering as dataset_filtering

In [2]:
git_repo_path = '/Users/dean/Documents/gitRepos'
gutenberg_repo_path = os.path.join(git_repo_path, 'gutenberg')
gutenberg_analysis_repo = os.path.join(git_repo_path, 'gutenberg-analysis')

In [3]:
## import internal helper functions
analysis_src_dir = os.path.join(gutenberg_analysis_repo,'src')
sys.path.append(analysis_src_dir)
from data_io import get_book

gutenberg_src_dir = os.path.join(gutenberg_repo_path,'src')
sys.path.append(gutenberg_src_dir)

from metaquery import meta_query
from jsd import jsdalpha

# Read in both metadata files

In [4]:
mq_filepath=os.path.join(gutenberg_repo_path,'metadata','metadata.csv')
pg_catalog_filepath=os.path.join(git_repo_path, 'gutenberg_corpus_analysis', 'sample_dataset', 'pg_catalog.csv')

# Load both  metadata files

Load both the metadata file generated by SPGC and the metadata file from PG

In [5]:
df = dataset_filtering.read_metadata_and_catalog(mq_filepath, pg_catalog_filepath)
original_shape=df.shape

Get only English books, according to PG catalog

In [6]:
df = df.query('Language=="en"')
df['Language'].unique()

array(['en'], dtype=object)

Let's verify that the language column in both metadata files match

In [7]:
df['language'].unique()

array(["['en']", "['ne']"], dtype=object)

Uh oh, it doesn't!  What book is this causing the problems?

In [8]:
df.query('Language=="en" and language=="[\'ne\']"')

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,PG_ID,type_pgc,title_pgc,Language,Authors,Subjects,LoCC,Bookshelves
72655,PG75522,The soldier's orphans,"Stephens, Ann S. (Ann Sophia)",1810.0,1886.0,['ne'],0,set(),Text,75522,Text,The soldier's orphans,en,"Stephens, Ann S. (Ann Sophia), 1810-1886",American fiction -- 19th century,PS,NaN


Lets get rid of it!

In [9]:
index_to_drop = df.query('Language=="en" and language=="[\'ne\']"').index
df.drop(index_to_drop, inplace=True)

In [10]:
# Verifying that everything is good
print(df['Language'].unique())
print(df['language'].unique())

['en']
["['en']"]


In [11]:
print(f'Original Shape: {original_shape}')
print(f'Current Shape: {df.shape}')

Original Shape: (74141, 17)
Current Shape: (58991, 17)


# Lets get rid of anything missing a title or an author

In [12]:
tdf = df[['title', 'title_pgc', 'author', 'Authors']]
tdf[tdf.isnull().any(axis=1)]

,title,title_pgc,author,Authors
30,Spalding's Official Baseball Guide - 1913,Spalding's Official Baseball Guide - 1913,NaN,"Foster, John B. (John Buckingham), 1863-1941 [..."
93,Moorish Literature: Comprising Romantic Ballad...,Moorish Literature\r\nComprising Romantic Ball...,NaN,"Basset, René, 1855-1924 [Editor]"
114,"The Great Events by Famous Historians, Volume ...","The Great Events by Famous Historians, Volume ...",NaN,"Johnson, Rossiter, 1840-1931 [Editor]; Horne, ..."
126,"The Great Events by Famous Historians, Volume ...","The Great Events by Famous Historians, Volume ...",NaN,"Johnson, Rossiter, 1840-1931 [Editor]; Horne, ..."
134,The Literature of Arabia: With Critical and Bi...,The Literature of Arabia\r\nWith Critical and ...,NaN,"Wilson, Epiphanius, 1845-1916 [Editor]"
...,...,...,...,...
75226,Who Was Who: 5000 B. C. to Date: Biographical ...,Who Was Who: 5000 B. C. to Date\r\nBiographica...,NaN,"Gordon, Irwin Leslie, 1888-1954 [Editor]"
75300,Spalding's Baseball Guide and Official League ...,Spalding's Baseball Guide and Official League ...,NaN,"Chadwick, Henry, 1824-1908 [Editor]"
75305,The Garden of Bright Waters: One Hundred and T...,The Garden of Bright Waters\r\nOne Hundred and...,NaN,"Mathers, E. Powys (Edward Powys), 1892-1939 [T..."
75314,"The Great Events by Famous Historians, Volume 12","The Great Events by Famous Historians, Volume 12",NaN,"Johnson, Rossiter, 1840-1931 [Editor]; Horne, ..."


Well, it looks like most of these HAVE authors (or at least editors), it's just messed up in the metadata created by SPGC.  Let's just drop them.

In [13]:
tdf.head()

,title,title_pgc,author,Authors
0,The Magna Carta,The Magna Carta,Anonymous,Anonymous
1,Apocolocyntosis,Apocolocyntosis,"Seneca, Lucius Annaeus","Seneca, Lucius Annaeus, 5? BCE-65; Rouse, W. H..."
2,The House on the Borderland,The House on the Borderland,"Hodgson, William Hope","Hodgson, William Hope, 1877-1918"
3,"My First Years as a Frenchwoman, 1876-1879","My First Years as a Frenchwoman, 1876-1879","Waddington, Mary King","Waddington, Mary King, 1833-1923"
4,The Warriors,The Warriors,"Lindsay, Anna Robertson Brown","Lindsay, Anna Robertson Brown, 1864-1948"


In [14]:
to_drop = tdf[tdf.isnull().any(axis=1)].index
df.loc[to_drop].head()

,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,PG_ID,type_pgc,title_pgc,Language,Authors,Subjects,LoCC,Bookshelves
30,PG10028,Spalding's Official Baseball Guide - 1913,NaN,NaN,NaN,['en'],125,{'Baseball'},Text,10028,Text,Spalding's Official Baseball Guide - 1913,en,"Foster, John B. (John Buckingham), 1863-1941 [...",Baseball,GV,Browsing: Sports/Hobbies/Motoring
93,PG10085,Moorish Literature: Comprising Romantic Ballad...,NaN,NaN,NaN,['en'],186,{'Spanish literature -- Translations into Engl...,Text,10085,Text,Moorish Literature\r\nComprising Romantic Ball...,en,"Basset, René, 1855-1924 [Editor]",Spanish literature -- Translations into English,PQ,Browsing: Literature
114,PG10103,"The Great Events by Famous Historians, Volume ...",NaN,NaN,NaN,['en'],157,{'World history'},Text,10103,Text,"The Great Events by Famous Historians, Volume ...",en,"Johnson, Rossiter, 1840-1931 [Editor]; Horne, ...",World history,D,Browsing: History - European; Browsing: Histor...
126,PG10114,"The Great Events by Famous Historians, Volume ...",NaN,NaN,NaN,['en'],273,{'World history'},Text,10114,Text,"The Great Events by Famous Historians, Volume ...",en,"Johnson, Rossiter, 1840-1931 [Editor]; Horne, ...",World history,D,Greece; Browsing: History - European; Browsing...
134,PG10121,The Literature of Arabia: With Critical and Bi...,NaN,NaN,NaN,['en'],109,{'Arabic literature'},Text,10121,Text,The Literature of Arabia\r\nWith Critical and ...,en,"Wilson, Epiphanius, 1845-1916 [Editor]",Arabic literature,PJ,Browsing: Culture/Civilization/Society; Browsi...


In [15]:
df.drop(to_drop, inplace=True)

# Lets see if titles match

In [16]:
dont_match, attribute_errors = dataset_filtering.compare_columns(df, 'title', 'title_pgc', verbose=True)#['author']

Dont Match: id: PG59774   Thirty Strange Stories   30 Strange Stories
Dont Match: id: PG63765   The Divine and Perpetual Obligation of the Observance of the Sabbath, with Reference more Especially to a Pamphlet Lately Puvblished by the Rev. C.J. Vaughan, D.D., Head Master of Harrow School, Entitled “A Few Words on the Crystal Palace Question”   Mutiny
Dont Match: id: PG6420   Copyright Renewals 1960   U.S. Copyright Renewals, 1960


Let's get rid of PG63765 and lets note that we should get rid of copyright renewals en masse.

We can also ditch the duplicate column

In [17]:
df = df[df['id']!='PG63765']
df.drop('title_pgc', axis=1, inplace=True)

# Verify Author Matches

In [18]:
dont_match, attribute_errors = dataset_filtering.compare_columns(df, 'author', 'Authors')
dont_match[['title','author','Authors']]

,title,author,Authors
1,Apocolocyntosis,"Seneca, Lucius Annaeus","Seneca, Lucius Annaeus, 5? BCE-65; Rouse, W. H..."
2,The House on the Borderland,"Hodgson, William Hope","Hodgson, William Hope, 1877-1918"
3,"My First Years as a Frenchwoman, 1876-1879","Waddington, Mary King","Waddington, Mary King, 1833-1923"
4,The Warriors,"Lindsay, Anna Robertson Brown","Lindsay, Anna Robertson Brown, 1864-1948"
5,A Voyage to the Moon: With Some Account of the...,"Tucker, George","Tucker, George, 1775-1861"
...,...,...,...
75389,"France and England in North America, Part III:...","Parkman, Francis","Parkman, Francis, 1823-1893"
75390,Poems,"Betham, Matilda","Betham, Matilda, 1776-1852"
75391,"Harriet, the Moses of Her People","Bradford, Sarah H. (Sarah Hopkins)","Bradford, Sarah H. (Sarah Hopkins), 1818-1912"
75393,Collected Articles of Frederick Douglass,"Douglass, Frederick","Douglass, Frederick, 1818-1895"


Note that there are actually a bunch that don't match properly, but it appears that it is mostly a formatting issue.  We can come back to it, if needed.  Leaving them here

Lets ditch the duplicate authors column though

In [19]:
df.drop('Authors', axis=1, inplace=True)

# Do IDs match?  They better!

Note: This should be totally unnecessary since we joined on ID

In [20]:
# Create a numeric version of the 'id' column with the "PG" removed
df['id_numeric'] = (
    df['id']
    .str.replace('PG', '')  # remove the literal "PG"
    .astype(str)            # convert to integer
)

df['PG_ID'] = df['PG_ID'].astype(str)              # Convert numeric to string

dont_match, attribute_errors = dataset_filtering.compare_columns(
    df,
    'id_numeric',
    'PG_ID',
    verbose=True
)

In [21]:
print(dont_match)
print(attribute_errors)

Empty DataFrame
Columns: [id, title, author, authoryearofbirth, authoryearofdeath, language, downloads, subjects, type, PG_ID, type_pgc, Language, Subjects, LoCC, Bookshelves, id_numeric]
Index: []
[]


No entries show up with unmatching id's. We can drop the placeholder id_numeric column. I keep the redundant PG_ID column here not knowing if it will be useful later to query the raw data. 

# Where do we stand?

In [22]:
df.shape

(56712, 16)

In [23]:
df.columns

Index(['id', 'title', 'author', 'authoryearofbirth', 'authoryearofdeath',
       'language', 'downloads', 'subjects', 'type', 'PG_ID', 'type_pgc',
       'Language', 'Subjects', 'LoCC', 'Bookshelves', 'id_numeric'],
      dtype='object')

# Add information on the length of books

Adds the number of lines, the number of words, and the number of unique words

By default, drops the books you haven't downloaded

#### Add the total word count of the entry, called 'word_count'.

In [24]:
count_path = os.path.join(gutenberg_repo_path, 'data', 'counts')
df['word_count'] = df['id'].apply(lambda pid: dataset_filtering.get_word_count(pid, count_path))

#### Add the total unique word count of the entry, called 'unique_word_count'.

In [25]:


count_path = os.path.join(gutenberg_repo_path, 'data', 'counts')
df['unique_word_count'] = df['id'].apply(
    lambda pid: dataset_filtering.get_unique_word_count(pid, count_path)
)

#### Add total lines of text in the raw text file, called 'line_count'.

Note that this is taking line count of the somewhat-cleaned files in the text folder, not the files in the raw folder.

In [26]:


text_path = os.path.join(gutenberg_repo_path, 'data', 'text')
df['line_count'] = df['id'].apply(
    lambda pid: dataset_filtering.get_line_count(pid, text_path)
)

#### Add total tokens in the entry, called 'token_count'.

In [27]:


token_path = os.path.join(gutenberg_repo_path, 'data', 'tokens')
df['token_count'] = df['id'].apply(
    lambda pid: dataset_filtering.get_token_count(pid, token_path)
)

In [28]:
df.shape

(56712, 20)

# Drop Missing Books

In [32]:
# Drop rows with NaN values in the line_count column
df.dropna(subset=['line_count'], inplace=True)
df.shape

(30591, 20)

# Looking at Authors

In [33]:
df['author'].value_counts()

author
Various                                                   1934
Anonymous                                                  335
Oliphant, Mrs. (Margaret)                                  119
Jacobs, W. W. (William Wymark)                              99
Shakespeare, William                                        96
                                                          ... 
Gilchrist, Anne (Anne Burrows)                               1
Moore, Walter W. (Walter William)                            1
Miller, William J. (William John)                            1
California. Alaska-Yukon-Pacific exposition commission       1
Blancké, Wilton W. (Wilton Wallace)                          1
Name: count, Length: 12803, dtype: int64

In [36]:

#fig, ax = plt.subplots()
#df['author'].value_counts().plot(ax=ax, kind='bar')

In [ ]:
#df['author'].value_counts().plot(kind='bar')

It will be difficult to categorize "Various", "Anonymous", or "Unknown" authors, let's ditch them

In [37]:
df = df[~df['author'].isin(['Various', 'Anonymous', 'Unknown'])]
df.shape

(28281, 20)

Lets see how many authors have more than 10 or 20 books

In [38]:
vc = df['author'].value_counts()
vc

author
Oliphant, Mrs. (Margaret)                                 119
Jacobs, W. W. (William Wymark)                             99
Shakespeare, William                                       96
Balzac, Honoré de                                          88
Library of Congress. Copyright Office                      63
                                                         ... 
Gilchrist, Anne (Anne Burrows)                              1
Moore, Walter W. (Walter William)                           1
Miller, William J. (William John)                           1
California. Alaska-Yukon-Pacific exposition commission      1
Blancké, Wilton W. (Wilton Wallace)                         1
Name: count, Length: 12800, dtype: int64

In [39]:
print(f'There are a total of {len(vc)} authors')
for book_count in [10, 20, 30, 40, 50, 75, 100]:
    print(f'There are {len(vc[vc > book_count])} authors with more than {book_count} books')


There are a total of 12800 authors
There are 389 authors with more than 10 books
There are 128 authors with more than 20 books
There are 59 authors with more than 30 books
There are 31 authors with more than 40 books
There are 15 authors with more than 50 books
There are 4 authors with more than 75 books
There are 1 authors with more than 100 books


In [40]:
# What should be the minimum number of books per author?
book_count_cutoff=30

In [41]:
authors_to_include = vc[vc > book_count_cutoff].index

mask = df['author'].isin(authors_to_include)
df = df[mask]

In [42]:
df.shape

(2742, 20)

# SETTINGS

In [43]:
df['author'].value_counts().min()

31

In [44]:
#####################
#####################
def normalize_dataset(df, how='num_books'):
    author_list = df['author'].unique()
    vc = df['author'].value_counts()

    min_num_books = vc.min()

    for author in author_list:
        tdf = df.query('author==@author')
        num_to_drop = tdf.shape[0] - min_num_books
        ind_to_drop = tdf.sample(num_to_drop).index
        df.drop(ind_to_drop, inplace=True)
    
def split_test_train(df, train_perc=0.8):
    author_list = df['author'].unique()
    train_ind = []
    for author in author_list:
        tdf = df.query('author==@author')
        single_author_train_ind = tdf.sample(frac=train_perc).index
        train_ind = [*train_ind,*single_author_train_ind]

    train_df = df.loc[train_ind]
    test_df = df.drop(train_ind)

    return train_df, test_df
    

In [45]:
normalize_dataset(df)

In [46]:
df.shape

(1829, 20)

In [ ]:
train_df, test_df = split_test_train(df)

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
def write_csv_in_metadata_format(df, outfile):
    cols_to_keep = ['id', 'title', 'author', 'authoryearofbirth', 'authoryearofdeath',
       'language', 'downloads', 'subjects']
    df = df[cols_to_keep]
    df.to_csv(outfile)

In [ ]:
train_outfile = 'train.csv'
test_outfile = 'test.csv'
write_csv_in_metadata_format(train_df, train_outfile)
write_csv_in_metadata_format(test_df, test_outfile)